In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 133.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam
from google.colab import drive
drive.mount('/content/drive')
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10
DATASET_PATH = '/content/drive/MyDrive/Brain Tumor Segmentation/Training'
CATEGORIES = ['glioma', 'meningioma', 'notumor', 'pituitary']
data = []
labels = []
for category in CATEGORIES:
    folder_path = os.path.join(DATASET_PATH, category)
    print(f"Loading {category}...")
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        try:
            image = cv2.imread(img_path)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            data.append(image)
            labels.append(category)
        except Exception as e:
            pass
data = np.array(data) / 255.0
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), epochs=EPOCHS, verbose=1)
def predict_image(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    label = lb.inverse_transform(prediction)
    return label[0]
interface = gr.Interface(fn=predict_image,
                         inputs=gr.Image(type='pil'),
                         outputs=gr.Label(num_top_classes=4),
                         title="Brain Tumor Classifier",
                         description="Upload an image to classify a brain tumor as glioma, meningioma, notumor, or pituitary.")
interface.launch()


Mounted at /content/drive
Loading glioma...
Loading meningioma...
Loading notumor...
Loading pituitary...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.6252 - loss: 0.9173 - val_accuracy: 0.2747 - val_loss: 6.5255
Epoch 2/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7632 - loss: 0.6254 - val_accuracy: 0.5083 - val_loss: 3.3550
Epoch 3/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7896 - loss: 0.5390 - val_accuracy: 0.5678 - val_loss: 1.8157
Epoch 4/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8230 - loss: 0.4725 - val_accuracy: 0.4873 - val_loss: 1.5822
Epoch 5/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8386 - loss: 0.4051 - val_accuracy: 0.5862 - val_loss: 1.1850
Epoch 6/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8562 - loss: 0.3917 - val_accuracy: 0.5669 - val_loss: 1.2828
Epoch 7/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8718 - loss: 0.3448 - val_accuracy: 0.5066 - val_loss: 3.4700
Epoch 8/10
286/286 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8702 - loss: 0.3554 - val_acc